In [119]:
from math import *
import matplotlib.pyplot as plt
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 20}

matplotlib.rc('font', **font)

# reading in dicom files
import pydicom
# ipywidgets for some interactive features
from ipywidgets.widgets import * 
import ipywidgets as widgets

#module to test
import Utility

In [120]:
#test center of mass multiplication
# 0 divisions

print(Utility.CenterOfMassMultiplication.__doc__)
A = np.zeros((2,2,3))
#print(np.max.__doc__)
print(np.maximum(1.0,0.0))
B = Utility.CenterOfMassMultiplication(A, axis = None)
print(np.sum(A))
print(np.maximum(1, np.sum(A)))
print(B)
if (np.all(B == 0)):
    print("test passed")

calculate Center of mass of numpy array with array multiplication using array coordinates. 

    Center of mass is calculated for dimensions specified with axis, allowing vectorization. 

    make sure that the array datatype has no overflow issues with np.sum(arr). 

    return: coordinates / np.maximum(1, counts)
1.0
0.0
1.0
[0. 0. 0.]
test passed


In [121]:
#test center of mass on a small array
A = np.ones((2,3,5,7))
#print("Array A\n",A)
com = Utility.CenterOfMassMultiplication(A)
print("center of mass of A\n",com)
print("com =", com)
if (com.shape[0] == np.ndim(A)):
    print("test passed!!!")

center of mass of A
 [0.5 1.  2.  3. ]
com = [0.5 1.  2.  3. ]
test passed!!!


In [122]:
#test selective axis center of mass
com = Utility.CenterOfMassMultiplication(A, axis =(0,2))
#print("center of mass of A\n",com)
#print("com shape", com.shape)
#print("com [:,0,0]", com[:, 0,0])
if (np.all([com.shape[0]==2, com.shape[1]==A.shape[1], com.shape[2]==A.shape[3]])):
    print("test passed!!!")

test passed!!!


In [117]:
#Thank you Siemens 4 ring mCT walkthrough
def binAddresToProjectionSpace(event):
    """ extract [tofbin,mi,tx,robin] bin using formula: event = 1*robin+c1*tx+c1*c2*mi+c1*c2*c3*tof + prompt + tag 
   robin = ro + 200"""
    # tag = bitExtracted(event,1,32)
    #prompt_bit = bitExtracted(event,1,31)
    event = np.array(event)
    bin_address_remains = event & 0X3FFFFFFF
    coefficients = [MICHELOGRAM_SIZE, SINOGRAM_SIZE, NUMBER_OF_PROJECTIONS, 1 ]
    coordinates = np.zeros((len(coefficients),np.size(event)), dtype=int)
    for k in range(0, len(coefficients)):
        coeff = coefficients[k]
        coordinates[k] = bin_address_remains // coeff
        bin_address_remains -= coordinates[k]*coeff
    #coordinates[-1] -= NUMBER_OF_PROJECTIONS // 2 #reintroduce negative range
    #coordinates = np.flip(coordinates, axis=0)
    coordinates = np.squeeze(coordinates)
    return coordinates

def projectionSpaceToBinAddres(coordinates):
    """ convert coordinates [ro, tx, mi, tof] to 1D bin address"""
    temp = np.multiply(coordinates, [1,
                                        NUMBER_OF_PROJECTIONS, 
                                        SINOGRAM_SIZE, 
                                        MICHELOGRAM_SIZE])
    s = np.sum(temp)
    return int(s + NUMBER_OF_PROJECTIONS//2)

In [ ]:
def viewNDImages(images, fixedDimensions, fixedValues,  subplotShape = None, 
                 titles=None, axisLabels=None, figsize=None):
    """ plots 2D slices of N-dimensional images in subplots at position fixedValue of image dimensions fixedDimensions. """
    #print("fixD", fixedDimensions)
    #print("fixV", fixedValues)
    #print("axis labels", axisLabels)
    
    #try to find a nice way to grid them
    #assume images have the same size
    images = np.array(images)
    #make sure the array is sorted
    index_array = np.argsort(fixedDimensions)
    #print("index ar", (np.array(index_array),))
    fixedDimensions = np.array(fixedDimensions)[index_array]
    fixedValues = np.array(fixedValues)[index_array]
    #change axis labels correspondingly
    if axisLabels is None:
        axisLabels = [None, None]
    
    #determine which axes to move to the back, per image
    freeDimensions = np.ones((images.ndim-1,), dtype=int)
    freeDimensions[list(fixedDimensions)] = 0
    source = np.nonzero(freeDimensions)[0] + 1
    destination = (-2,-1)
    
    # " Move axes of an array to new positions.
    # Other axes remain in their original order. "
    images = np.moveaxis(a=images, source=source,destination=destination)
    plt.figure(figsize=figsize)
    if subplotShape is None:
        subplotShape = [len(images), 1]
    for k in range(len(images)):
        plt.subplot(subplotShape[0], subplotShape[1], k+1)
        plt.imshow(images[(k,) + tuple(fixedValues)])
        plt.xlabel(axisLabels[1])
        plt.ylabel(axisLabels[0])
        if titles is not None:
            plt.title(titles[k])
            
def interactHistograms(histograms, titles=None, subplotShape=None, figsize=None):
    """ assumes a sequence of 4D histograms [tof, mi, tx, ro] """
    #determine view
    #slice dimension 1
    #slice dimension 2
    dimensions = np.arange(4)
    horizontalWidget = widgets.IntSlider(min=0,max=4-1, value=3,
                                         description='horiz. axis')
    verticalWidget = widgets.IntSlider(min=0, max=4-1, value=2, description = 'vert. axis')
    #check difference with an if statement
    
    #determine position on fixed dimensions
    # for example mi, tof
    
    positionWidgetA = widgets.IntSlider(min=0)
    positionWidgetB = widgets.IntSlider(min=0)
    
    #update
    def fixedDimensions(horiz, vert):
        dimensions = np.arange(4)
        #print("bFixed2D", [dimensions != horiz, dimensions != vert])
        #print("bFixed", np.all([dimensions != horiz, dimensions != vert], axis=0))
        bFixedDims = np.all([dimensions != horiz, dimensions != vert], axis=0)
        fixedDims = dimensions[bFixedDims]
        return fixedDims
    
    def updatePositionWidgets(*args):
        fixedDims = fixedDimensions(verticalWidget.value, horizontalWidget.value)
        for k, widg in enumerate([positionWidgetA, positionWidgetB]):
            dim = fixedDims[k]
            widg.max = HISTOGRAM_SHAPE[dim] -1
            widg.description = HISTOGRAM_DIMENSION_TITLES[dim]
            
    for widg in [verticalWidget, horizontalWidget]:
        widg.observe(updatePositionWidgets, 'value')
    
    def f(vDim, hDim, posA,posB):
        if vDim != hDim:
            fixedDims =fixedDimensions(vDim, hDim)
            #print(fixedDims)
            viewNDImages(images=histograms, fixedDimensions=fixedDims, fixedValues=[posA, posB],
                                subplotShape=subplotShape, titles = titles,
                                axisLabels=[HISTOGRAM_DIMENSION_TITLES[hDim],HISTOGRAM_DIMENSION_TITLES[vDim]],
                                            figsize=figsize)
        else: 
            print("warning: non-distinct vertical and horizontal axes")
    
    interact(f, vDim=verticalWidget, hDim=horizontalWidget, posA=positionWidgetA, posB=positionWidgetB);
    
def interactImages(images, titles=None, subplotShape=None, figsize=None):
    """ assumes a sequence of 3D images [z,y,x] """
    #determine view
    #fixed dimension
    dimensions = np.arange(3)
    fixedDimensionWidget = widgets.IntSlider(min=0, max=dimensions[-1], value=0)
    
    #determine position on fixed dimension
    # for example z
    
    positionWidget = widgets.IntSlider(min=0)
    
    #updates
    def updatePositionWidget(*args):
        positionWidget.max = IMAGE_SHAPE[fixedDimensionWidget.value] -1
        positionWidget.description = IMAGE_DIMENSION_TITLES[fixedDimensionWidget.value]
            
    fixedDimensionWidget.observe(updatePositionWidget, 'value')
    updatePositionWidget()
    
    def f(fixDim, fixPos):
            #print(fixedDims)
            #determine image axes from fixdim
            dimensions = np.arange(3)
            hDim,vDim = dimensions[dimensions != fixDim]
            
            viewNDImages(images=images, fixedDimensions=[fixDim], fixedValues=[fixPos],
                                subplotShape=subplotShape, titles = titles,
                                axisLabels=[IMAGE_DIMENSION_TITLES[hDim],IMAGE_DIMENSION_TITLES[vDim]],
                                            figsize=figsize)
    
    interact(f, fixDim=fixedDimensionWidget, fixPos=positionWidget);

In [7]:
def isEvent(event):
    tag = (event >> 31) & 1
    if (tag == 0): #event tag
        return True
        # prompt_bit = (event >> 30) & 1
        # if (prompt_bit== 0): #delayed
        # else: # coincidence
    else: #packet
        #if (((event >> 28) & 0Xe) == 8): #elapsed time tag 
        #if (( event >> 24 & 0XFF) == 0XC4): #horizontal bed position 
        #if ((event >> 24 & 0XFC) == 0XBC): #lost event tally
        return False
    
def timeSlices(dataLM, timeTagsPerSlice=1):
    """ split Listmode data according to time tags """
    bTagAndTime = np.zeros((2, len(dataLM)), dtype=np.bool)
    bTagAndTime[0] = ((dataLM >> 31) & 1) == 1
    bTagAndTime[1] = ((dataLM >> 28) & 0Xe) == 8
    timeTags = np.nonzero(np.all(bTagAndTime, axis=0))[0]
    
    splits = timeTags[timeTagsPerSlice: :timeTagsPerSlice]
    return np.split(dataLM, splits)

def events(dataLM):
    """ takes numpy array of 32 bit listmode data and only returns the events (that is, tags removed)"""
    return dataLM[((dataLM >> 31) & 1) == 0]

def delaysOfEvents(events):
    """ takes numpy array of 32 bit listmode data with events and only returns the delays (that is, prompts removed)"""
    return events[(events >> 30) & 1 == 0]

def bDelays(events):
    """ a bool array indicating delays in LM events"""
    return (events >> 30) & 1 == 0

def promptsOfEvents(events):
    """ takes numpy array of 32 bit listmode data with events and only returns the prompts (that is, delays removed)"""
    return events[(events >> 30) & 1 == 1]

def bPrompts(events):
    """ a bool array indicating prompts in LM events"""
    return (events >> 30) & 1 == 1

def binAddressFromLM(LM):
    """ extract the bin adress bits of LM events """
    return LM & 0X3FFFFFFF 
    
def get2DMichelogramIndex(michelogram_index):
    """ converts 1D michelogram index to segment number (tilt) and axial offset (halfplanes). \n precalculated segment_index_map and axial_index_map in Utility are faster """
    #mi / seg_table[0]
    # 55 + 55 -1 
    segment_bins = np.cumsum(SEGMENT_TABLE)
    segment_number = np.searchsorted(segment_bins, michelogram_index+1, side='left', sorter=None)
    
    
    if segment_number == 0:
        axial_offset = michelogram_index 
    else:
        segment_index = michelogram_index - segment_bins[segment_number-1]
        axial_offset = segment_index + (SEGMENT_TABLE[0] - SEGMENT_TABLE[segment_number])//2 
    
    return segment_number, axial_offset

def binaryString(number):
    return "{0:b}".format(number)

def listmodePathToHistogram(listmode_path):
    """ reads listmode data from listmode_path and returns a 4D numpy array with the histogram. \n
    the dimensions are transaxial angle, radial offset, michelogram index, and tof bin"""
    #read
    with open(listmode_path, 'rb') as file:
        listmode_data = np.fromfile(file, dtype=np.uint32)
    #Extract event data
    event_data = listmode_data[((listmode_data >> 31) & 1) == 0]
    #extract position in histogram
    bin_addresses=event_data & 0X3FFFFFFF 
    #construct histogram
    histogram = np.zeros((PROJECTION_SPACE_SIZE,),dtype=np.ushort)
    np.add.at(histogram, bin_addresses,1) #histogram[bin_addresses] += 1 # does not work as expected
    
    return np.reshape(histogram, HISTOGRAM_SHAPE)



    
def bPointInFOV(point3D):
    for k, pos in enumerate(point3D):
        if pos < 0:
            return False
        if pos > IMAGE_SHAPE[k]-1:
            return False
    return True



def bValidImageBin(points3D):
    """ vectorized function to check if bins are valid for image array ( not less than 0 or too large )"""
    criterions = np.zeros((2*3,np.size(points3D[0])), dtype=np.bool)
    for k in range(3):
        criterions[2*k] = points3D[k] > -1
        criterions[2*k+1] = points3D[k] < IMAGE_SHAPE[k]
    return np.all(criterions, axis=0)

def bValidBin(points, shapeFOV):
    """ generalized version of bValidImageBin, checks if bins are valid for array (not less than 0 or to large for shape) """
    ndims = len(shapeFOV)
    criterions = np.zeros((2*ndims,np.size(points[0])), dtype=np.bool)
    for k in range(ndims):
        criterions[2*k] = points[k] > -1
        criterions[2*k+1] = points[k] < shapeFOV[k]
    return np.all(criterions, axis=0)

#thank you Dicom tutorial
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in               
              os.listdir(path)]
    slices = [s for s in slices if 'SliceLocation' in s]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] -   
                          slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - 
                      slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices

#thank you Dicom tutorial
def get_pixels_hu(scans):
    #get image
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    #print(np.min(image))
    #print(np.max(image))
    
    #image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    
    #intercept = scans[0].RescaleIntercept
    #slope = scans[0].RescaleSlope
    
    #if slope != 1:
    #    image = slope * image.astype(np.float64)
    #   image = image.astype(np.int16)
        
    #image += np.int16(intercept)
    return np.array(image, dtype=np.int16)

In [118]:
def imageBinFromHistogramBin(histogramBin, offsetTOF=False, delays=False, dtype=int, verbose=False):
    """ input is a sequence [tofbin,mi,tx,ro]. return the image bin [z,y,x]. Is vectorized. 
    the use of tof offset, delays, and datatype is optional"""
    #origin (Siemens) position in array
    tofbin, mi,tx, robin = histogramBin

    #reintroduce negative ro
    ro = robin - NUMBER_OF_PROJECTIONS//2
    
    ao = AXIAL_OF_MI[mi]
    seg = SEGMENT_OFFSET_MAP[SEGMENT_OF_MI[mi]]
    if delays:
        tof = np.zeros_like(tofbin, dtype=int)
        prompts = tofbin != len(TOF_OFFSET_MAP)
        delays = prompts == False
    
        tof[prompts] = TOF_OFFSET_MAP[tofbin[prompts]]
        tof[delays] = 1 + len(TOF_OFFSET_MAP)//2 
    else:
        tof = TOF_OFFSET_MAP[tofbin]
    

    #constants
    origin_array = [0, IMAGE_SHAPE[1]//2, IMAGE_SHAPE[2]//2]
    #header file info, relating bins and mm
    #norm /sino / LM file hdr
    mm_per_pixel = np.array([2.027, 4.07283, 4.07283]) 
    mm_per_ro = 2.005
    # header file
    # "transaxial FOV diameter (cm) :=70.4"
    # mCT brochure / Jakoby et al
    # "the industry’s only 78 cm bore ""

    # e7tools vg60 gm_check 1104 (model 1104)
    # ...
    # crystalRadius()=42.76cm
    # ... 
    # sinogramDepthOfInteraction()=0.67cm
    # LORDepthOfInteraction()=0.96cm
    # ...
    #  tofOffset()=0.039ns
    # tofBinSize()=0.3125ns
    # ... 

    #cylinder_radius_mm = 78*10/2
    crystalRadius_mm = 42.76*10
    LORDepthOfInteraction_mm= 9.6
    sinogramDepthOfInteraction_mm=6.7
    cylinderRadius_mm = crystalRadius_mm+LORDepthOfInteraction_mm
    cylinderRadius_mm_square = np.power(cylinderRadius_mm, 2)
    tof_bin_time_s = 312.5*1e-12
    tof_offset_s = tof_bin_time_s / 8.0
    c_lightspeed_m_per_s = 299792458

    #calculate in mm space first, then convert to pixels
    # assume Siemens coordinates, then translate by origin to array
    #decide directions R,S,T from angles
    #then no worries about ro == 0
    #As R,S are 2D and transaxial, only T changes between pixel space and mm space
    #tof bin requires mm space
    Rx = TX_ANGLE_TO_RX[tx]
    Ry = TX_ANGLE_TO_RY[tx]
    if np.any(Ry < 0):
        print("invalid R direction, y < 0")
    
    Sx = Ry
    Sy = -1*Rx
    if np.any(Sx < 0):
        print("invalid S direction, x < 0")
    
    S_direction = np.array([Sy, Sx])

    
    # now calculate the detector points A,B of the LOR, and the point C between them, starting with C
    C_point_mm = np.array([ao*mm_per_pixel[0],
                             ro*Ry*mm_per_ro,
                             ro*Rx*mm_per_ro])
    ro_mm_square = np.power(ro*mm_per_ro, 2)
    
    CA_mm_square = cylinderRadius_mm_square-ro_mm_square
    if (np.any(CA_mm_square < 0)):
        print("negative CA_mm_square")
        print("cylinderRadius_mm_square", cylinderRadius_mm_square)
        print("ro_mm_square", ro_mm_square)
    
    CA_mm = np.sqrt(CA_mm_square)
    rd_mm = seg*AXIAL_COMPRESSION*mm_per_pixel[0]
    
    #determine position A,B by z0, C, and CB,CA in figure above
    # A,B not explicitly used, but useful for eventual correction
    #A_position_mm = np.array(C_point_mm)
    #A_position_mm[1:] -= CA_mm*S_direction
    #B_position_mm = C_point_mm
    #B_position_mm[1:] += CA_mm*S_direction
    #A_position_mm[0] -= rd_mm  
    #B_position_mm[0] += rd_mm
    # assume A,B positions are distinct
    
    #determine axis T
    T_direction_mm = np.zeros_like(C_point_mm)
    T_direction_mm[1:] = CA_mm*S_direction
    T_direction_mm[0] = rd_mm
    
    #normalize T
    norms = np.linalg.norm(T_direction_mm, axis=0)

    if np.any(norms == 0):
        print("invaliv normalization 0")
    T_direction_mm = T_direction_mm / norms
    if np.any(T_direction_mm[2] < 0):
        print("invalid T direction, x < 0")
    

    #correction term tof_offset_s should be negative according to documentation
    t_s = tof*tof_bin_time_s 
    if offsetTOF:
        t_s += tof_offset_s
    t_mm = t_s*c_lightspeed_m_per_s*1e3
    #convert T_direction (in pixel space) to direction in mm space
    t_point_mm = t_mm*T_direction_mm
    
    event_point_mm = t_point_mm + C_point_mm     
    
    if verbose:
        def shape(arr):
            return np.array(arr).shape
        
        print("--- Shapes ---")
        print("event_point_mm", shape(event_point_mm))
        print("origin_array", shape(origin_array))
    
    event_img_bin = []
    # do it dimension by dimension to avoid shape issues
    for k in range(3):
        event_img_bin.append(event_point_mm[k] / mm_per_pixel[k] + origin_array[k])
        
    return np.array(event_img_bin, dtype=dtype)

In [ ]:
#test binAddresToProjectionSpace
ro= 2
tx = 3
mi = 5
tof = 7
event = ro*1+200 + tx*Utility.NUMBER_OF_PROJECTIONS + mi*Utility.SINOGRAM_SIZE+tof*Utility.MICHELOGRAM_SIZE
coordinates = Utility.binAddresToProjectionSpace(event)
print("difference", np.array([ro,tx,mi,tof])-np.array(coordinates))

#test array compatibility
events = [event, event+1000]
coordinates = Utility.binAddresToProjectionSpace(events)
print("coordinates of 2 events\n", coordinates)

In [ ]:
#print attributes
for atr in inspect.getmembers(Utility):
    print(atr)

In [ ]:
# test bitExtracted
number = 171
k = 5
p = 1
print(k, "bits from bit ", p, " in ", number )
print ("The extracted number is (decimal)", Utility.bitExtracted(number, k, p) )
print("input number (binary): ", "{0:b}".format(number))
print("output number (binary)", "{0:b}".format(Utility.bitExtracted(number, k, p)))
#print("{0:b}".format((1 << 6)-1))
#print("{0:b}".format(number >> (p)))

In [2]:
#test projectionSpaceToBinAddres 
ew = int(23)
print(ew)
coords = Utility.binAddresToProjectionSpace(ew)
print(coords)
result = Utility.projectionSpaceToBinAddres(coords)
print(result)
print("difference", ew - result)

23


NameError: name 'Utility' is not defined

In [ ]:
#test get2DMichelogramIndex
print("michelogram index, segment")
seg = -1
for k in range(0,sum(Utility.SEGMENT_TABLE)):
    temp, _ = Utility.get2DMichelogramIndex(k)
    if (temp > seg):
        seg = temp
        print(k, seg)

In [ ]:
#test get2DMichelogramIndex
axial_offset = []
segment = []
x = []
for k in range(0,sum(Utility.SEGMENT_TABLE)):
    seg, ao = Utility.get2DMichelogramIndex(k)
    segment.append(seg)
    axial_offset.append(ao)
    x.append(k)

plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(x,segment)
plt.ylabel("segment")
plt.xlabel("michelogram index")
plt.subplot(2,1,2)
plt.plot(x,axial_offset)
plt.ylabel("axial offset")
plt.xlabel("michelogram index")
plt.show()

In [ ]:
#compare get2DMichelogramIndex with precalculated functions
inconsistency_count = 0
for k in range(0, Utility.NUMBER_OF_SINOGRAMS):
    seg, ao = Utility.get2DMichelogramIndex(k)
    if (seg != Utility.SEGMENT_INDEX_MAP[k]):
        print("segment inconsistencies at mi ", k)
        inconsistency_count +=1
    if (ao != Utility.AXIAL_INDEX_MAP[k]):
        print("axial offset inconsistencies at mi ", k)
        inconsistency_count +=1
print("number of inconsistencies:", inconsistency_count)

In [ ]:
#test histogram
path = r"C:/Users/petct/Desktop/CylinderPET-Converted/CylinderPET-LM-00/CylinderPET-LM-00.l"
histogram = Utility.listmodePathToHistogram(path)

In [ ]:
#test center of mass
CenterOfMass = CenterOfMassMultiplication(histogram_4D)
print("radial offset, transaxial angle, michelogram index, tof bin")
print(CenterOfMass)

In [ ]:
print(com)
print(com.shape)
print(histogram.shape)

In [ ]:
print(tuple(np.arange(0,len(Utility.HISTOGRAM_SHAPE),1)))
tu = tuple(np.arange(0,len(Utility.HISTOGRAM_SHAPE),1))
#tu[2] = 23
print(tu)
A = np.ones((3))
print(A)
A[ [s for s in (0,2)]]=23
print(A)
print(A[[a for a in (True,True,False)]])

In [ ]:
#test mi_from_seg
target = np.arange(Utility.NUMBER_OF_SINOGRAMS)
result = np.array([])
for k in range(len(Utility.SEGMENT_TABLE)):
    result = np.append(result, Utility.mi_from_seg(k))

diff = target - result
print("difference count=", np.sum(diff))

In [2]:
#test mi_bounds_from_seg

for k in range(len(Utility.SEGMENT_TABLE)):
    target = Utility.mi_from_seg(k)
    mi_low, mi_high = Utility.mi_bounds_from_seg(k)
    result = np.arange(mi_low,mi_high)
    diff = target - result
    print("segment", k, "difference count=", np.sum(diff))

segment 0 difference count= 0
segment 1 difference count= 0
segment 2 difference count= 0
segment 3 difference count= 0
segment 4 difference count= 0
segment 5 difference count= 0
segment 6 difference count= 0
segment 7 difference count= 0
segment 8 difference count= 0


In [ ]:
#test mi_from_seg_ao()
error_count = 0
for k in range(Utility.NUMBER_OF_SINOGRAMS):
    seg = Utility.SEGMENT_INDEX_MAP[k]
    ao = Utility.AXIAL_INDEX_MAP[k]
    estimate = Utility.mi_from_seg_ao(seg, ao)
    if (estimate != k):
        print("error at sinogram", k)
        error_count+=1
print("error_count=", error_count)